In [13]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from mtcnn import MTCNN  # Thư viện phát hiện khuôn mặt

In [ ]:
# Định nghĩa kích thước hình ảnh
IMG_SIZE = (128, 128)  # Kích thước đầu vào cho mô hình

# Load mô hình đã huấn luyện
model = load_model("face_recognition_model.h5")
# model = load_model("face_recognition_model.h5")

# Danh sách các nhãn
# class_names = [
#     'DoMixi', 
#     'HariWon', 
#     'HieuThuHai', 
#     'HoNgocHa', 
#     'karik', 
#     'MisThy', 
#     'PhuongAnhDao', 
#     'SonTungMTP', 
#     'ThuyTien', 
#     'TranThanh'
# ]
class_names = [
    'DoMixi', 
    'HariWon', 
    'HieuThuHai', 
    'HoNgocHa', 
    'Misthy', 
    'PhuongAnhDao', 
    'SonTungMTP', 
    'ThuyTien', 
    'TranThanh', 
    'karik'
]
# Ngưỡng xác định "Unknown" (ví dụ: nếu độ tin cậy < 50%)
CONFIDENCE_THRESHOLD = 50.0

In [135]:
# Đường dẫn video
video_path = r"C:\DL\NDKM\video\karik.mp4"

# Mở video
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(video_path)

# Khởi tạo bộ phát hiện khuôn mặt MTCNN
detector = MTCNN()

# Tăng tốc độ phát video (số frame bị bỏ qua giữa mỗi lần xử lý)
frame_skip = 5
frame_count = 0

In [136]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Bỏ qua frame để tăng tốc
    if frame_count % frame_skip != 0:
        continue

    # Phát hiện khuôn mặt trên frame
    results = detector.detect_faces(frame)
    
    for result in results:
        # Lấy vị trí khuôn mặt
        x, y, width, height = result['box']
        x, y = max(0, x), max(0, y)  # Đảm bảo không âm
        face = frame[y:y+height, x:x+width]  # Crop khuôn mặt

        # Resize khuôn mặt về kích thước chuẩn
        try:
            resized_face = cv2.resize(face, IMG_SIZE)
        except Exception as e:
            print(f"Resize error: {e}")
            continue
        normalized_face = resized_face / 255.0
        reshaped_face = np.expand_dims(normalized_face, axis=0)

        # Dự đoán nhãn
        predictions = model.predict(reshaped_face, verbose=0)
        confidence = np.max(predictions) * 100  # Tính toán độ tin cậy
        if confidence < CONFIDENCE_THRESHOLD:
            predicted_label = "Unknown"
        else:
            predicted_label = class_names[np.argmax(predictions)]

        # Vẽ khung quanh khuôn mặt
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(frame, f'{predicted_label} ({confidence:.2f}%)', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Hiển thị kết quả
    cv2.imshow("Face Recognition", frame)

    # Thoát khi nhấn phím 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()